In [52]:
import torch
import sae_lens

In [82]:
model_name = "pythia-70m-deduped"
layers = 6
release = "pythia-70m-deduped-res-sm"
sae_id = f"blocks.0.hook_resid_pre"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = sae_lens.HookedSAETransformer.from_pretrained(model_name)
sae = sae_lens.SAE.from_pretrained(release=release, sae_id=sae_id, device=device)[0]

Loaded pretrained model pythia-70m-deduped into HookedTransformer


In [83]:
# Ensure dim_0 and dim_1 are defined
dim_0, dim_1 = sae.W_dec.shape
sae = sae_lens.SAE.from_pretrained(release=release, sae_id=sae_id, device=device)[0]
with torch.no_grad():
    sae.W_dec.requires_grad = False
    print(sae.W_dec.nonzero().shape)
    sae.W_dec[0, :].zero_()
    print(sae.W_dec.nonzero().shape)

torch.Size([16777216, 2])
torch.Size([16776704, 2])


In [69]:
acts = torch.load('../res/math_pythia_70m_deduped_res_sm_freq_mean_global.pt')

/tmp/ipykernel_4204/2843757290.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  acts = torch.load('../res/math_pythia_70m_deduped_res_sm_freq_mean_global.pt')


In [93]:
acts.numel()/7

32768.0

In [94]:
top_1_percent_values, top_1_percent_indices = torch.topk(
    acts, int(1e-4 * acts.numel()//7), largest=True
)

In [96]:
top_1_percent_indices

tensor([[22576, 23714, 19106],
        [    2,     1,     0],
        [25962, 11194, 16477],
        [ 7754, 28827,  6845],
        [ 9977, 28951, 28262],
        [20290, 27648, 20259],
        [ 1990,  7541, 23666]], device='cuda:0')

In [105]:
sae = sae_lens.SAE.from_pretrained(release=release, sae_id=sae_id, device=device)[0]
top_1_percent_indices = top_1_percent_indices.to(sae.W_dec.device)

with torch.no_grad():
    sae.W_dec.requires_grad = False
    list(map(lambda idx: sae.W_dec[idx, :].zero_(), top_1_percent_indices[0]))
    print(sae.W_dec[top_1_percent_indices[0][0], :])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [75]:
with torch.no_grad():
    sae.W_dec[layer_0_freq, :].zero_()